# Preparing the Data for Modeling:


## Index:
- [Importing the Data](#Importing)
- [Data Preprocessing](#DPP): Shifting the Data and Feature Engineering
    - [Shifting the Dates & Feature Engineering](#Shifting)
- Preparing a Train and Test Set
    - [Manually Splitting the Data](#Splitting) to Predict 2017 & onwards.
----------

## Importing Libraries:

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

-----

## Company Name
**Company Options:**

- Apple, Inc. - `Apple`
- Facebook, Inc. -`Facebook`
- Google LLC - `Google`
- JPMorgan Chase & Co. - `JPMorgan`
- The Goldman Sachs Group, Inc. - `GoldmanSachs`
- Moody's Corporation - `Moodys`
- The International Business Machines Corporation (IBM) - `IBM`
- Twitter Inc. - `Twitter`
- BlackRock, Inc. - `BlackRock`
- Microsoft Corporation - `Micrisoft`

In [2]:
company_name = 'Apple'

<a class="anchor" id="Importing"></a>
## Importing the Original Dataset:
The Original dataset will be imported.

In [3]:
def data_importer(company_name):
    """ 
    Returns the clean Data corresponding to the company; the data is imported from a CSV. 
    Additionally, the Date Column is converted to date time and placed as the index.
    
    Parameter
    -------------------------------------------------------------------------------------------------------------
    company_name : str
        Passes the string with the company's name.
    """
    company_name=company_name
    # Reading from a CSV into a pandas dataframe.
    df = pd.read_csv(f'data/{company_name}_Clean.csv')
    # Turning the Date column into an actual date time.
    df['Date'] = pd.to_datetime(df.Date)
    # Setting the Date as the Index.
    df.set_index('Date', inplace=True)
    # Sorting the Index into an acsending order.
    df.sort_index(inplace=True, ascending=True)
    return df

In [4]:
df = data_importer(company_name)

### Taking a look at the Original Dataframe:

In [5]:
df.head(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.402563,0.402563,0.400652,0.400652,43971200.0
1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.373010,0.373010,0.371246,0.371246,26432000.0


-----
<a class="anchor" id="DPP"></a>
# Data Preprocessing

### Creating a Function to Shift the Date One Business Day into the Future:
The date is being shifted one day into the future in order to forecast the last day.

In [6]:
def date_shifter(df):
    """ 
    Returns a dataframe that has shifted a business day into the future.
    
    Parameter
    ----------
    df : var
        Passes the variable assigned to a dataframe.
    """
    shifted_df = pd.DataFrame(df[:-1].values, index = df[1:].index, columns=df.columns)
    return shifted_df

<a class="anchor" id="Shifting"></a>
### Creating a Function to Feature Engineer and Introduce Lag:

Feature Engineering Moving Averages:


**Moving Averages** (MA) are indicators that follow trends, or lags, based on past prices often filtering and smoothening any "noise" from random price fluctuations. Two types of moving averages are calculated for this stock: Simple Moving Average & Exponential Moving Average. Each is calculated with a short (12-Day), medium (26-Day), and long (85-Day) lag.

- **Simple Moving Averages (SMA)**: is calculated by adding the most recent prices in a given period and dividing that value by the days in the period; therefore, computing the average.
- **Exponential Moving Averages (EMA)**: similar to the SMA, but an exponential weight is applied to all observations in a period of time.

Additionally, the date will be shifted using the shifter function.

In [7]:
def lag_and_shift_data(dataframe):
    """ 
    
    Returns a shifted data frame with two engineered moving averages: Simple & Exponential. 
    Each type of moving average is calculated with a short (12-Day), medium (26-Day), and long (85-Day) lag.
    The first few weeks with null values are dropped.
    
    Functions
    ------------------------------------------------------------------------------------------------
    date_shifter :  var = pd.DataFrame()
        The data frame is shifted using the shift function, which shifts the frame one day into the future.
    
    
    """
    temp_df = dataframe.copy()
    
    # Engineering the Simple Moving Averages:
    short_SMA = temp_df.rolling(window=12).mean().copy()
    mid_SMA = temp_df.rolling(window=26).mean().copy()
    long_SMA = temp_df.rolling(window=85).mean().copy()

    # Engineering the Exponential Moving Averages:
    short_EMA = temp_df.ewm(span=12, adjust=False).mean().copy()
    mid_EMA = temp_df.ewm(span=26, adjust=False).mean().copy()
    long_EMA = temp_df.ewm(span=85, adjust=False).mean().copy()
    
    # Merging the Simple Moving Average dataframes with the original data frame.
    temp_df = pd.merge(temp_df, short_SMA, left_index=True, right_index=True, suffixes=['','_Short_SMA'])
    temp_df = pd.merge(temp_df, mid_SMA, left_index=True, right_index=True, suffixes=['','_Mid_SMA'])
    temp_df = pd.merge(temp_df, long_SMA, left_index=True, right_index=True, suffixes=['','_Long_SMA'])
    
    # Merging the Exponential Moving Average data frames with the original data frame.
    temp_df = pd.merge(temp_df, short_EMA, left_index=True, right_index=True, suffixes=['','_Short_EMA'])
    temp_df = pd.merge(temp_df, mid_EMA, left_index=True, right_index=True, suffixes=['','_Mid_EMA'])
    temp_df = pd.merge(temp_df, long_EMA, left_index=True, right_index=True, suffixes=['','_Long_EMA'])
    
    # Dropping the First Few Weeks W/ Null Values:
    temp_df.dropna(inplace=True) 
    
    # Shifting the Data frame One Day into the Future:
    temp_df = date_shifter(temp_df)

    return temp_df

### Creating the New Data Set with Time-Shift and Lag:

In [8]:
df_shift = lag_and_shift_data(df)

### Saving the Shifted Dataframe:

In [9]:
# Saving the df_shift
df_shift.to_csv(f'../stocks/data/modeling_data/{company_name}_Shifted_DF.csv', index=True)

### Inspecting the Time-Shifted Data Set:

In [10]:
df_shift.head(2)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
1981-04-15,27.88,28.00,27.88,27.88,29700.0,0.0,1.0,0.409914,0.411679,0.409914,...,27.100545,27.100545,395955.964908,0.0,1.0,0.399428,0.400780,0.398454,0.398454,2.217353e+07
1981-04-16,26.63,26.63,26.50,26.50,152000.0,0.0,1.0,0.391536,0.391536,0.389625,...,27.086579,27.086579,390282.570375,0.0,1.0,0.399245,0.400565,0.398249,0.398249,2.185582e+07


In [11]:
df_shift.tail(2)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2018-03-26,168.39,169.92,164.94,164.94,40248954.0,0.0,1.0,168.39,169.92,164.94,...,169.581218,170.954500,3.373761e+07,0.000506,1.0,171.062781,172.593499,169.561596,170.934731,3.373761e+07
2018-03-27,168.07,173.10,166.44,172.77,36272617.0,0.0,1.0,168.07,173.10,166.44,...,169.508167,170.996721,3.379657e+07,0.000494,1.0,170.993182,172.605278,169.489000,170.977412,3.379657e+07


-------
<a class="anchor" id="Splitting"></a>
# Preparing a Train and Test Set:

In [12]:
def data_splitter(dataframe, end_date, start_date):
    """
    Returns an train and test data set according to the dates passed.
    
    Parameter
    ---------------
    dataframe : var
        Passes a variable assigned to a pandas data frame.
    
    end_date : str
        Passes a date in the form of a string. The train set ends on the date provided.
        String formatted as >> 'YYYY-MM-DD'
    
    start_date : str
        Passes a date in the form of a string. The test set begins on the date provided.
        String formatted as >> 'YYYY-MM-DD'
    """
    temp_df = dataframe.copy()
    # Splitting the data: the train set ends on the date provided & the test set starts at the date provided/
    X_train, X_test = temp_df[:end_date], temp_df[start_date:]
   
    return X_train, X_test

## Splitting the Data to Predict 2017 & Onwards:

In [13]:
X_train, X_test = data_splitter(df_shift, '2016-12-30', '2017-01-03')

### Taking a look at the Training Set:

In [14]:
X_train.tail(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-28,116.52,117.8000,116.49,117.26,18296855.0,0.0,1.0,115.080808,116.344998,115.051178,...,110.566050,111.471650,3.338105e+07,0.007306,1.0,109.570218,110.508678,108.902823,109.794971,3.338106e+07
2016-12-29,117.52,118.0166,116.20,116.76,20905892.0,0.0,1.0,116.068456,116.558923,114.764760,...,110.697072,111.594635,3.309093e+07,0.007136,1.0,109.721340,110.649382,109.039147,109.923409,3.309094e+07
2016-12-30,116.45,117.1095,116.40,116.73,15039519.0,0.0,1.0,115.011672,115.663027,114.962290,...,110.829698,111.714062,3.267113e+07,0.006970,1.0,109.844371,110.765978,109.176895,110.048172,3.267114e+07


### Taking a look at the Test Set:

In [15]:
X_test.head(3)

,Open,High,Low,Close,Volume,Ex_Dividend,Split_Ratio,Adj_Open,Adj_High,Adj_Low,...,Low_Long_EMA,Close_Long_EMA,Volume_Long_EMA,Ex_Dividend_Long_EMA,Split_Ratio_Long_EMA,Adj_Open_Long_EMA,Adj_High_Long_EMA,Adj_Low_Long_EMA,Adj_Close_Long_EMA,Adj_Volume_Long_EMA
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,116.65,117.20,115.43,115.82,30586265.0,0.0,1.0,115.209202,115.752409,114.004271,...,110.936682,111.809549,3.262264e+07,0.006808,1.0,109.969134,110.881942,109.289159,110.149132,3.262266e+07
2017-01-04,115.80,116.33,114.76,116.15,28781865.0,0.0,1.0,114.369701,114.893155,113.342546,...,111.025596,111.910490,3.253332e+07,0.006650,1.0,110.071473,110.975226,109.383424,110.255324,3.253334e+07
2017-01-05,115.85,116.51,115.75,116.02,21118116.0,0.0,1.0,114.419083,115.070931,114.320318,...,111.135466,112.006060,3.226785e+07,0.006495,1.0,110.172580,111.070475,109.498236,110.356060,3.226787e+07


### Saving the Train & Test Set into a CSV:

In [16]:
# Saving the X_train
X_train.to_csv(f'../stocks/data/modeling_data/{company_name}_X_Train.csv', index=True)

In [17]:
# Saving the X_test
X_test.to_csv(f'../stocks/data/modeling_data/{company_name}_X_Test.csv', index=True)